**An Automated Discharge Summary system Built for
Multiple Clinical Texts by Pre-trained distilbart Model**

The code mounts Google Drive into the Colab environment, allowing access to files and folders stored in the Google Drive.

Installind and importing necessary libraries

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from transformers import BartTokenizer, TFBartForConditionalGeneration
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split

2024-02-23 14:15:27.841029: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-23 14:15:27.877635: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-23 14:15:27.877673: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-23 14:15:27.878777: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-23 14:15:27.885540: I tensorflow/core/platform/cpu_feature_guar

In [2]:
from datasets import load_metric
from tqdm import tqdm
import numpy as np
import nltk
nltk.download("punkt")
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize


[nltk_data] Downloading package punkt to /home/vs428/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/vs428/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
from datasets import Features, Value

The code reads a CSV dataset named "sample_data_1000.csv" from the mounted Google Drive and loads it into a variable called "Dataset_for_BART_Model." Then, it uses the Hugging Face's `load_dataset` function to load the same CSV dataset into a variable called "ds."

In [4]:
Dataset_for_BART_Model = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/combined_data_export_with_prompts.csv')
ds = load_dataset('csv',
                    data_files = '/gpfs/milgram/project/rtaylor/shared/DischargeMe/combined_data_export_with_prompts.csv',
                 # features=Features({
                 #     "idx":Value(dtype="int32", id=None),
                 #     "hadm_id":Value(dtype="int32", id=None),
                 #     "brief_hospital_course":Value(dtype="string", id=None),
                 #     "prompts":Value(dtype="string", id=None),
                 # })
                 )


Found cached dataset csv (/home/vs428/.cache/huggingface/datasets/csv/default-0ee6477ca1b61895/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
Dataset_for_BART_Model.columns

Index(['idx', 'hadm_id', 'brief_hospital_course', 'prompts'], dtype='object')

In [6]:
def convert_to_string(example):
  for key in example.keys():
    example[key] = str(example[key])
  return example

ds = ds.map(convert_to_string)

Loading cached processed dataset at /home/vs428/.cache/huggingface/datasets/csv/default-0ee6477ca1b61895/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-01a4263d58b6ac36.arrow


In [7]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")
model = AutoModelForSeq2SeqLM.from_pretrained("sshleifer/distilbart-cnn-12-6")

In [8]:
model_name = "sshleifer/distilbart-cnn-12-6"

In [9]:
# import torch
# dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# model.to(dev)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerN

The code imports a configuration class called "BartConfig" from the Hugging Face's Transformers library. It then creates a BART model configuration by loading a pre-trained configuration named "philschmid/tf-distilbart-cnn-12-6." The code accesses the maximum input dimension of the BART model configuration and stores it in the variable "max_input_dimension."

In [10]:
from transformers import BartConfig

# Create a BART model configuration
config = BartConfig.from_pretrained("sshleifer/distilbart-cnn-12-6")

# Access the maximum input dimension
max_input_dimension = config.max_position_embeddings

print("Maximum Input Dimension:", max_input_dimension)

Maximum Input Dimension: 1024


In [11]:
# tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})


Preprocessing dataset

In [12]:
remove_columns = ["hadm_id"]

ds = ds["train"].remove_columns(remove_columns)


In [13]:

max_input_length = 256 #400 #4000 #47000
max_target_length = 256 #200 #1000 #10700
#(10681.578, 46985.783)
from nltk.corpus import stopwords
from nltk.corpus import stopwords

def preprocess_function(examples):
    stop_words = set(stopwords.words('english')) # Get the English stop words

    # Define a function to remove stop words from a given text
    def remove_stop_words(text):
        tokens = text.split()
        filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
        return ' '.join(filtered_tokens)

    # Remove stop words from examples['Concatenated_Text']
    examples['prompts'] = [remove_stop_words(text) for text in examples['prompts']]

    # Remove stop words from examples['Discharge summary']
    examples['brief_hospital_course'] = [remove_stop_words(text) for text in examples['brief_hospital_course']]

    model_inputs = tokenizer(
        examples['prompts'],
        max_length=max_input_length,
        truncation=True,
        padding=True,
        return_tensors="pt",
    )

    with tokenizer.as_target_tokenizer():
        labels=tokenizer(
            examples['brief_hospital_course'],
            max_length=max_target_length,
            truncation=True,
            padding=True,
            return_tensors="pt",
        )

    model_inputs["labels"] = labels["input_ids"]
    model_inputs["references"] = examples['brief_hospital_course']

    return model_inputs

tokenized_datasets = ds.map(preprocess_function, batched=True)

Parameter 'function'=<function preprocess_function at 0x151aefa46a70> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

/gpfs/milgram/project/rtaylor/vs428/conda_envs/dischargeme/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


The code sets the value of the test size to 15% of the total dataset size. Then, it processes the tokenized dataset and splits it into training and testing subsets using the specified test size.

In [14]:
test_size=.15

processed_dataset= tokenized_datasets.shuffle().train_test_split(test_size=test_size)

The code sets various configuration parameters for training a machine learning model using the Hugging Face Transformers library.

In [15]:
# import tensorflow as tf


# num_train_epochs = 5
# train_batch_size = 2
# eval_batch_size = 2
# learning_rate = 5.6e-5
# weight_decay_rate= 0.01
# num_warmup_steps= 155
# output_dir=model_name.split("/")[1]
# # hub_token = HfFolder.get_token() 
# # hub_model_id = f'{model_name.split("/")[1]}-tradetheevent'
# fp16= False #True


# if fp16:
#   tf.keras.mixed_precision.set_global_policy("mixed_float16")



The code imports a model class called "TFAutoModelForSeq2SeqLM" from the Hugging Face Transformers library. It then loads a pre-trained sequence-to-sequence language model using the "from_pretrained" method and stores it in the variable "model."

The code imports a data collator class called "DataCollatorForSeq2Seq" from the Hugging Face Transformers library. It creates a data collator instance named "data_collator" that dynamically pads the input and label sequences.

Then, it converts the processed training dataset ("processed_dataset['train']") and testing dataset ("processed_dataset['test']") into TensorFlow `tf.data.Dataset` objects. During conversion, it selects specific columns ("input_ids," "attention_mask," and "labels") from the dataset and shuffles the training dataset. It uses the previously created "data_collator" to collate the data into batches based on the specified batch sizes for training and evaluation ("train_batch_size" and "eval_batch_size," respectively).

In [16]:
import transformers
transformers.__version__

'4.30.2'

In [17]:
processed_dataset["train"]

Dataset({
    features: ['idx', 'brief_hospital_course', 'prompts', 'input_ids', 'attention_mask', 'labels', 'references'],
    num_rows: 425
})

In [18]:
processed_dataset

DatasetDict({
    train: Dataset({
        features: ['idx', 'brief_hospital_course', 'prompts', 'input_ids', 'attention_mask', 'labels', 'references'],
        num_rows: 425
    })
    test: Dataset({
        features: ['idx', 'brief_hospital_course', 'prompts', 'input_ids', 'attention_mask', 'labels', 'references'],
        num_rows: 75
    })
})

In [19]:
remove_columns = ['idx', 'brief_hospital_course', "prompts"]

processed_dataset = processed_dataset.remove_columns(remove_columns)


In [20]:
processed_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'references'],
        num_rows: 425
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'references'],
        num_rows: 75
    })
})

In [21]:
import datasets
datasets.__version__

# import transformers
# transformers.__version__

'2.13.0'

In [22]:
processed_dataset['train'].features#.to_tf_dataset

{'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'references': Value(dtype='string', id=None)}

In [23]:
processed_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'references'],
        num_rows: 425
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'references'],
        num_rows: 75
    })
})

In [25]:

from transformers import DataCollatorForSeq2Seq

# Data collator that will dynamically pad the inputs received, as well as the labels.
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# # converting our train dataset to tf.data.Dataset
# tf_train_dataset = processed_dataset["train"].to_tf_dataset(
#    columns=["input_ids", "attention_mask", "labels"],
#    shuffle=True,
#    batch_size=1,
#    collate_fn=data_collator
# )

# # converting our test dataset to tf.data.Dataset
# tf_eval_dataset = processed_dataset["test"].to_tf_dataset(
#    columns=["input_ids", "attention_mask", "labels"],
#    shuffle=True,
#    batch_size=1,
#    collate_fn=data_collator)


The code imports a function called "create_optimizer" from the Hugging Face Transformers library. It creates an optimizer with weight decay, which is commonly used for fine-tuning machine learning models.

The number of training steps is calculated based on the length of the training dataset and the number of training epochs. The optimizer is initialized with the specified learning rate, weight decay rate, and the number of warm-up steps.

After creating the optimizer and learning rate schedule, the model is compiled using the specified optimizer.

In [26]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer


In [27]:
batch_size = 1
# model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,

)

In [28]:
# from transformers import create_optimizer


# # create optimizer wight weigh decay
# num_train_steps = len(tf_train_dataset) * num_train_epochs
# optimizer,lr_schedule = create_optimizer(
#     init_lr=learning_rate,
#     num_train_steps=num_train_steps,
#     weight_decay_rate=weight_decay_rate,
#     num_warmup_steps=num_warmup_steps,
# )

# # compile model
# model.compile(optimizer=optimizer)


In this code, several callbacks are defined and stored in the "callbacks" list. First, the "TensorboardCallback" from TensorFlow is included to log training and evaluation metrics during model training. Then, an "if" condition checks if the "hub_token" variable has a value. If the condition is true (i.e., a valid Hugging Face Hub token is available), the "PushToHubCallback" from Hugging Face Transformers is included in the "callbacks" list. This callback is used to push the trained model and its associated tokenizer to the Hugging Face model hub, allowing easy sharing and version control of the model. The "PushToHubCallback" requires the "output_dir," "tokenizer," "hub_model_id," and "hub_token" as inputs, which are provided accordingly in the callback definition.

In [40]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    print(result)
    return {k: round(v, 4) for k, v in result.items()}

In [41]:
import os
from transformers.keras_callbacks import PushToHubCallback
from tensorflow.keras.callbacks import TensorBoard as TensorboardCallback

callbacks=[]

# callbacks.append(TensorboardCallback(log_dir=os.path.join(output_dir,"logs")))
# hub_token = False
# if hub_token:
#   callbacks.append(PushToHubCallback(output_dir=output_dir,
#                                      tokenizer=tokenizer,
#                                      hub_model_id=hub_model_id,
#                                      hub_token=hub_token))




Fit the model

In [42]:
tokenized_datasets

Dataset({
    features: ['idx', 'brief_hospital_course', 'prompts', 'input_ids', 'attention_mask', 'labels', 'references'],
    num_rows: 500
})

In [43]:
# import torch
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# model.to(device)

In [44]:
# train_results = model.fit(
#     tf_train_dataset,
#     validation_data=tf_eval_dataset,
#     callbacks=callbacks,
#     epochs=num_train_epochs,
# )

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [45]:
trainer.train()

Epoch,Training Loss,Validation Loss


{'rouge1': (Score(precision=0.2018864112967795, recall=0.13835394350674052, fmeasure=0.15438251913254905), Score(precision=0.23139345039353787, recall=0.15556272154181477, fmeasure=0.17283682014862428), Score(precision=0.2632967514717338, recall=0.17647090636046134, fmeasure=0.19562683011643586), Score(precision=0.2018864112967795, recall=0.13835394350674052, fmeasure=0.15438251913254905), Score(precision=0.23139345039353787, recall=0.15556272154181477, fmeasure=0.17283682014862428), Score(precision=0.2632967514717338, recall=0.17647090636046134, fmeasure=0.19562683011643586), Score(precision=0.2018864112967795, recall=0.13835394350674052, fmeasure=0.15438251913254905), Score(precision=0.23139345039353787, recall=0.15556272154181477, fmeasure=0.17283682014862428), Score(precision=0.2632967514717338, recall=0.17647090636046134, fmeasure=0.19562683011643586), Score(precision=0.2018864112967795, recall=0.13835394350674052, fmeasure=0.15438251913254905), Score(precision=0.23139345039353787

TypeError: type tuple doesn't define __round__ method

calculating ROUGE

In [51]:
tokenizer("question: What is 42 ? context: 42 is the answer to life, the universe and everything")

{'input_ids': [0, 40018, 35, 653, 16, 3330, 17487, 5377, 35, 3330, 16, 5, 1948, 7, 301, 6, 5, 9468, 8, 960, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [53]:
from transformers import pipeline
text2text_generator = pipeline(model=model, tokenizer=tokenizer)
text2text_generator()

RuntimeError: Inferring the task automatically requires to check the hub with a model_id defined as a `str`.BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)


In [46]:
outputs = trainer.predict(processed_dataset["test"])

{'rouge1': (Score(precision=0.2018864112967795, recall=0.13835394350674052, fmeasure=0.15438251913254905), Score(precision=0.23139345039353787, recall=0.15556272154181477, fmeasure=0.17283682014862428), Score(precision=0.2632967514717338, recall=0.17647090636046134, fmeasure=0.19562683011643586), Score(precision=0.2018864112967795, recall=0.13835394350674052, fmeasure=0.15438251913254905), Score(precision=0.23139345039353787, recall=0.15556272154181477, fmeasure=0.17283682014862428), Score(precision=0.2632967514717338, recall=0.17647090636046134, fmeasure=0.19562683011643586), Score(precision=0.2018864112967795, recall=0.13835394350674052, fmeasure=0.15438251913254905), Score(precision=0.23139345039353787, recall=0.15556272154181477, fmeasure=0.17283682014862428), Score(precision=0.2632967514717338, recall=0.17647090636046134, fmeasure=0.19562683011643586), Score(precision=0.2018864112967795, recall=0.13835394350674052, fmeasure=0.15438251913254905), Score(precision=0.23139345039353787

TypeError: type tuple doesn't define __round__ method

In [47]:
outputs

NameError: name 'outputs' is not defined

In [ ]:
metric = load_metric("rouge")

In [ ]:
# def evaluate(model, dataset):
#     all_predictions = []
#     all_labels = []
#     for batch in tqdm(dataset):
#         predictions = model.generate(batch["input_ids"])
#         decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
#         labels = batch["labels"].numpy()
#         labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#         decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
#         decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
#         decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
#         all_predictions.extend(decoded_preds)
#         all_labels.extend(decoded_labels)
#         result = metric.compute(
#             predictions=decoded_preds, references=decoded_labels, use_stemmer=True
#         )
#     result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
#     return {k: round(v, 4) for k, v in result.items()}

# # results = evaluate(model, processed_dataset["test"])


ROUGE scores

In [ ]:
print(results)
#1 epoch = {'rouge1': 34.0009, 'rouge2': 19.8162, 'rougeL': 27.9357, 'rougeLsum': 32.1849}